In [ ]:
from pathlib import Path
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')


In [ ]:
data_vacinas = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas.csv')
data_vacinas_detalhes = pd.read_csv(Path.cwd() / '..' / '..' /'vacinas_detalhe.csv')


In [ ]:
# ",".join(data_vacinas_detalhes.columns)


In [ ]:
df_vacinas = data_vacinas.copy()
df_vacinas.drop( [x for x in df_vacinas.columns if '_novas' in x or 'pessoas' in x], axis=1, inplace=True)
#df_vacinas.columns

df_vacinas_detalhes = data_vacinas_detalhes.copy()
df_vacinas_detalhes.drop( [x for x in df_vacinas_detalhes.columns if '_novas' in x or 'pessoas' in x], axis=1, inplace=True)
#df_vacinas_detalhes.columns

#df = df_vacinas_detalhes.merge(df_vacinas, how='left', on='data', suffixes=("_semanal", "_diario"))
df = df_vacinas.merge(df_vacinas_detalhes, how='left', on='data', suffixes=("_diario", "_semanal"))
df.ffill(inplace=True)
#",".join(df.columns)

for i in ['', '1', '2']:
    df[f'doses{i}_diff'] = df[f'doses{i}_continente'] - df[f'doses{i}_diario']
cols = ["data"] + [ f'doses{i}_{j}' for i in ['', '1', '2'] for j in ['diff', 'continente', 'diario']]

df[cols].tail(10)


In [ ]:
# ",".join(list(df.columns))


In [ ]:
# idades
df[ [col for col in df.columns if 'perc' in col][18:] ].tail(10)


In [ ]:
# ars
df[ [col for col in df.columns if 'perc' in col][:18] ].tail(10)


In [ ]:
# ars
dfl = df[ [col for col in df.columns if 'perc' in col][:18] ][-1:]

doses1, doses2 = float(dfl['doses1_perc']), float(dfl['doses2_perc'])
for region in ['arsnorte', 'arscentro', 'arslvt', 'arsalentejo', 'arsalgarve', 'madeira', 'açores']:
    d1, d2 = float(dfl[f'doses1_perc_{region}']), float(dfl[f'doses2_perc_{region}'])
    print(f"region={region} vacinados: {round( (d2-doses2)*100, 1)}% 1ª dose: {round((d1-doses1)*100, 1)}% ")

